<a href="https://colab.research.google.com/github/symphonynet/SymphonyNet/blob/main/play_symphonynet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Music with SymphonyNet

## Install the requirements

In [1]:
!git clone https://github.com/therohans/SymphonyNet.git

Cloning into 'SymphonyNet'...
remote: Enumerating objects: 143, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 143 (delta 24), reused 0 (delta 0), pack-reused 99
Receiving objects: 100% (143/143), 1.74 MiB | 10.23 MiB/s, done.
Resolving deltas: 100% (49/49), done.


In [ ]:
!cd SymphonyNet; git checkout getting_running; git pull origin getting_running

In [ ]:
!pip install torch==1.12.1

In [2]:
!pip install -r SymphonyNet/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 776.8 MB 18 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.7.1 which is incompatible.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.7.1 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.7.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 4.9 MB/s 
     |████████████████████████████████| 92 kB 12.7 MB/s 
    

## Model preparation

In [3]:
MAX_POS_LEN = 4096
PI_LEVEL = 2
IGNORE_META_LOSS = 1
RATIO = 4
BPE = "_bpe" # or ""

DATA_BIN=f"linear_{MAX_POS_LEN}_chord{BPE}_hardloss{IGNORE_META_LOSS}"
CHECKPOINT_SUFFIX=f"{DATA_BIN}_PI{PI_LEVEL}"
DATA_BIN_DIR=f"SymphonyNet/data/model_spec/{DATA_BIN}/bin/"
DATA_VOC_DIR=f"SymphonyNet/data/model_spec/{DATA_BIN}/vocabs/"
from SymphonyNet.src.fairseq.gen_utils import process_prime_midi, gen_one, get_trk_ins_map, get_note_seq, note_seq_to_midi_file, music_dict
music_dict.load_vocabs_bpe(DATA_VOC_DIR, 'SymphonyNet/data/bpe_res/' if BPE == '_bpe' else None)

Initialize the model and load pretrained parameters. (You should first save the provided [ckpt file](https://drive.google.com/file/d/1xpkj_qN4MdLRkBdCXmfGjuWWjnTN1Og0/view) into your google drive.)

In [4]:
from fairseq.models import FairseqLanguageModel
from google.colab import drive
drive.mount('/content/drive')
custom_lm = FairseqLanguageModel.from_pretrained('.', 
    checkpoint_file=f'drive/MyDrive/checkpoint_last_{CHECKPOINT_SUFFIX}.pt', 
    data_name_or_path=DATA_BIN_DIR, 
    user_dir="SymphonyNet/src/fairseq/linear_transformer_inference")

Mounted at /content/drive


In [5]:
import torch

m = custom_lm.models[0]
if torch.cuda.is_available():
    m.cuda()
m.eval()

## Prepare prime MIDI

In [6]:
midi_name = 'SymphonyNet/test.mid'
max_measure_cnt = 5
max_chord_measure_cnt = 0
prime, ins_label = process_prime_midi(midi_name, max_measure_cnt, max_chord_measure_cnt)

## Generation

In [7]:
import time
while(True):
  try:
    generated, ins_logits = gen_one(m, prime, MIN_LEN = 1024)
    break
  except Exception as e:
    print(e)
    continue
trk_ins_map = get_trk_ins_map(generated, ins_logits)
note_seq = get_note_seq(generated, trk_ins_map)
timestamp = time.strftime("%m-%d_%H-%M-%S", time.localtime()) 
output_name = f'output_prime{max_measure_cnt}_chord{max_chord_measure_cnt}_{timestamp}.mid'
note_seq_to_midi_file(note_seq, output_name)

 48%|████▊     | 1853/3895 [00:21<00:23, 86.54it/s]


## Audio Display

In [8]:
!DEBIAN_FRONTEND=noninteractive sudo apt install -y fluidsynth

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  fluid-soundfont-gm libfluidsynth1 libqt5x11extras5 qsynth
Suggested packages:
  fluid-soundfont-gs timidity jackd
The following NEW packages will be installed:
  fluid-soundfont-gm fluidsynth libfluidsynth1 libqt5x11extras5 qsynth
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 120 MB of archives.
After this operation, 150 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fluid-soundfont-gm all 3.1-5.1 [119 MB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libfluidsynth1 amd64 1.1.9-1 [137 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fluidsynth amd64 1.1.9-1 [20.7 kB]
Get:4 http://archi

In [9]:
!pip install --upgrade pyfluidsynth

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
!pip install pretty_midi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.6 MB 4.7 MB/s 
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-py3-none-any.whl size=5591955 sha256=33c63afe9e2ff4d80c388425b7725beec0749df870ac03baa48682388f4c47ed
  Stored in directory: /root/.cache/pip/wheels/ad/74/7c/a06473ca8dcb63efb98c1e67667ce39d52100f837835ea18fa
Successfully built pretty-midi


In [13]:
import fluidsynth
import pretty_midi
from IPython import display
_SAMPLING_RATE = 16000
def display_audio(pm: pretty_midi.PrettyMIDI, seconds=300):
  waveform = pm.fluidsynth(fs=_SAMPLING_RATE)
  # Take a sample of the generated waveform to mitigate kernel resets
  waveform_short = waveform[:seconds*_SAMPLING_RATE]
  return display.Audio(waveform_short, rate=_SAMPLING_RATE)

In [14]:
pm = pretty_midi.PrettyMIDI(output_name)
display_audio(pm)